<a href="https://colab.research.google.com/github/sajjadsehat/Big-Data/blob/main/spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 49 kB/s 
     |████████████████████████████████| 199 kB 46.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=3929d08ae746f5120fe7e7ec7a039888fe24303d12483897e1697707c39384f4
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark

<div dir=rtl>
 مجموعه داده زیر شامل 100000 نمونه ازاطلاعات شخصی افرادی است که طی سال های 2005  الی 2008 با ویژگی های نام،نام خانوادگی،جنسیت، تاریخ تولد وکد پستی است که طی سال های مذکور طی ثبت های تکراری ذخیره گشته است. حال طی این پروژه به دنبال آنیم تا جفت ثبت ها را که متعلق به یک شخص است بیابیم.

In [ ]:
!curl -L -o donation.zip https://bit.ly/1Aoywaq
!unzip donation.zip
!unzip 'block_*.zip'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   163  100   163    0     0   2063      0 --:--:-- --:--:-- --:--:--  2063
100 53.8M  100 53.8M    0     0  30.3M      0  0:00:01  0:00:01 --:--:-- 52.9M
Archive:  donation.zip
 extracting: block_10.zip            
 extracting: block_1.zip             
 extracting: block_2.zip             
 extracting: block_3.zip             
 extracting: block_4.zip             
 extracting: block_5.zip             
 extracting: block_6.zip             
 extracting: block_7.zip             
 extracting: block_8.zip             
 extracting: block_9.zip             
  inflating: documentation           
  inflating: frequencies.csv         
Archive:  block_6.zip
  inflating: block_6.csv             

Archive:  block_4.zip
  inflating: block_4.csv             

Archive:  block_7.zip
  inflating: block_7.csv             

Archive:  block_5.zip


In [ ]:
#load dataset
df0 = spark.read.csv("block*.csv")

In [ ]:
df0.count()

5749142

In [ ]:
df0.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)



In [ ]:
df0.show(10)

+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
|  _c0|  _c1|         _c2|         _c3|         _c4|         _c5|    _c6|   _c7|   _c8|   _c9|   _c10|    _c11|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
| 3148| 8326|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|14055|94934|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|33948|34740|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|  946|71870|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|64880|71676|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|  

In [ ]:
parsed = spark.read.csv("block*.csv", header=True, inferSchema=True, nullValue="?")

In [ ]:
parsed.printSchema()

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)
 |-- cmp_fname_c2: double (nullable = true)
 |-- cmp_lname_c1: double (nullable = true)
 |-- cmp_lname_c2: double (nullable = true)
 |-- cmp_sex: integer (nullable = true)
 |-- cmp_bd: integer (nullable = true)
 |-- cmp_bm: integer (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)



In [ ]:
parsed.show(10)

+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| 3148| 8326|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|14055|94934|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|33948|34740|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|  946|71870|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|64880|71676|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|25739|45991|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|  

<div dir=rtl>
پاسخ سوال دوم: اولا نام مربوط به هر فیچر و نوع دیتا فیچر مشخص گردید.ثانیا مقادیر ؟ به عنوان null شناسایی شده است. یعنی جایگزین گردید.


In [ ]:
parsed.groupBy('is_match').count().show()

+--------+-------+
|is_match|  count|
+--------+-------+
|    true|  20931|
|   false|5728201|
+--------+-------+



<div dir=rtl>
پاسخ سوال 3: با استفاده از این دستور ستون مربوط به طبقه بندی مربوط به is match مقادیر  true و false شمار گردیده است.
در واقع فراوانی مقادیر فوق در ستون is match  می باشد.


In [ ]:
summary = parsed.describe()
summary.show()

+-------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+
|summary|              id_1|              id_2|      cmp_fname_c1|      cmp_fname_c2|       cmp_lname_c1|       cmp_lname_c2|            cmp_sex|             cmp_bd|             cmp_bm|            cmp_by|            cmp_plz|
+-------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+
|  count|           5749132|           5749132|           5748125|            103698|            5749132|               2464|            5749132|            5748337|            5748337|           5748337|            5736289|
|   mean| 33324.48559643438| 66587.43558331935|0.7129024704425707| 0.900017671890335|0.3156278193076

<div dir=rtl>
پاسخ سوال 4: مقدار count نشان دهنده مقادیر موجود است، دلیل متفاومت بودن آن این است که مقادیر پوچ شمرده نشده اند.

In [ ]:
matches = parsed.where("is_match = true")
match_summary = matches.describe()

misses = parsed.filter(col("is_match") == False) 
miss_summary = misses.describe()

In [ ]:
# Fill the null values  with 0
parsed.na.fill(value=0).show()


+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| 3148| 8326|         1.0|         0.0|         1.0|         0.0|      1|     1|     1|     1|      1|    true|
|14055|94934|         1.0|         0.0|         1.0|         0.0|      1|     1|     1|     1|      1|    true|
|33948|34740|         1.0|         0.0|         1.0|         0.0|      1|     1|     1|     1|      1|    true|
|  946|71870|         1.0|         0.0|         1.0|         0.0|      1|     1|     1|     1|      1|    true|
|64880|71676|         1.0|         0.0|         1.0|         0.0|      1|     1|     1|     1|      1|    true|
|25739|45991|         1.0|         0.0|         1.0|         0.0|      1|     1|     1|     1|      1|  